In [30]:
# Setting up the environment.
import numpy as np
import pandas as pd
from scipy import stats
import plotly as pl

In [36]:
# Load the data from the John Hopkins github repo
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-19-2020.csv')

In [37]:
# Dropping some columns and sorting

df1 = df[["Admin2", "Province_State", "Country_Region", "Confirmed", "Deaths", "Combined_Key"]] #getting the columns I want
df1 = df1[df1['Confirmed'] !=0] #dropping States with 0 confirmed
df1 = df1[(df1["Country_Region"] == "US")] #dropping countries other than the US
df1 = df1.sort_values(by=['Province_State','Confirmed'], ascending=True) #sorting by State and then Confirmed
df1 = df1.dropna() #dropping NA values
df1 = df1[df1.Province_State != 'Wuhan Evacuee'] #dropping this row because it is not US
df1['Combined_Key'] = df1['Combined_Key'].str.replace(r', US', '') #removing US from Combined key so it looks better in the hover text
df1 = df1.rename(columns={'Province_State': 'State'})
df1 = df1.rename(columns={'Admin2': 'County'})
df1 = df1.rename(columns={'Country_Region': 'Country'})
df1 = df1.rename(columns={'Combined_Key': 'County/State'})
df1.head(5)

,County,State,Country,Confirmed,Deaths,County/State
797,Fayette,Alabama,US,4,0,"Fayette, Alabama"
900,Geneva,Alabama,US,4,0,"Geneva, Alabama"
594,Crenshaw,Alabama,US,6,0,"Crenshaw, Alabama"
1359,Lamar,Alabama,US,8,0,"Lamar, Alabama"
270,Bullock,Alabama,US,9,0,"Bullock, Alabama"


In [33]:
import plotly.express as px
fig = px.treemap(df1, path=['State', 'County'], 
                 values='Confirmed',
                 names='State',
                 hover_name="County/State", 
                 #hover_data=["Confirmed", "Deaths","County/State"],
                
                 labels={'County', 'Confirmed'},
                 #title='Number of confirmed COVID19 cases'
                )

fig.update_layout(uniformtext=dict(minsize=18, mode=False),
                 hoverlabel=dict(bgcolor='white', bordercolor='black',
                      font=dict(family='Arial',
                          size=18,
                          color='darkgray')
                             ),
)

fig.show()

import plotly.io as pio
pio.write_html(fig, file='Index.html', auto_open=True)

In [38]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

levels = ['County', 'State', 'Country'] # levels used for the hierarchical chart
color_columns = ['Confirmed', 'Deaths']
value_column = 'Confirmed'

def build_hierarchical_dataframe(df1, levels, value_column, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy,
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'total'
        df_tree['value'] = dfg[value_column]
        df_tree['color'] = dfg[color_columns[0]] / dfg[color_columns[1]]
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id='total', parent='',
                              value=df[value_column].sum(),
                              color=df[color_columns[0]].sum() / df[color_columns[1]].sum()))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees


df_all_trees = build_hierarchical_dataframe(df1, levels, value_column, color_columns)
average_score = df1['Deaths'].sum() / df1['Confirmed'].sum()

fig = make_subplots(1, 2, specs=[[{"type": "domain"}, {"type": "domain"}]],)

fig.add_trace(go.Treemap(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    marker=dict(
        colors=df_all_trees['color'],
        colorscale='RdBu',
        cmid=average_score),
    hovertemplate='<b>%{label} </b> <br> Deaths: %{value}<br> Fatality rate: %{color:.2f}',
    name=''
    ), 1, 1)

fig.add_trace(go.Treemap(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    marker=dict(
        colors=df_all_trees['color'],
        colorscale='RdBu',
        cmid=average_score),
    hovertemplate='<b>%{label} </b> <br> Deaths: %{value}<br> Fatalityrate: %{color:.2f}',
    maxdepth=2
    ), 1, 2)

fig.update_layout(margin=dict(t=10, b=10, r=10, l=10))
fig.show()

KeyError: 'County'